In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]


In [5]:
folder = "../../Data/hpc_out/rwm_wp_2_6r7m/"

id_index = 1
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, "_")[id_index]) ? parse(Int, split(file, "_")[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" input output
        id = parse(Int, split(file, "_")[id_index])
        string = "$(id): $(input["T"]) | $(output.algorithm_measures["αs"][end]) | $(output.energy_measures["Es"][1]), $(minimum(output.energy_measures["Es"])), $(output.energy_measures["Es"][end])"
        evaluation_strings[id] = string
        min_Es[id] = minimum(output.energy_measures["Es"])
    end
end

In [16]:
function interactive_simulation_analysis(input, output)
    dgms = output.energy_measures["IDGMs"]
    Es = output.energy_measures["Es"]
    persistence_weight = -0.1
    realizations = [MorphoMol.Utilities.get_matrix_realization(state, input["template_centers"]) for state in output.states]
    tps = [MorphoMol.Energies.get_total_persistence(dgm) for dgm in dgms]
    cdgms2 = [dgm[2] for dgm in dgms]
    cdgms3 = [dgm[3] for dgm in dgms]
    cp2s = [MorphoMol.Energies.get_persistence(dgm) for dgm in cdgms2]
    cp3s = [MorphoMol.Energies.get_persistence(dgm) for dgm in cdgms3]
    mol_type = input["mol_type"]
    exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
    exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
    thetas = [MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, state) for state in output.states]
    xs = [i for i in 1:length(dgms)];
    two_ps = [MorphoMol.Energies.get_persistence(dgm[2], -0.1) for dgm in dgms]
    three_ps = [MorphoMol.Energies.get_persistence(dgm[3], -0.1) for dgm in dgms]
    tps = [MorphoMol.Energies.get_total_persistence(dgm, -0.1) for dgm in dgms];

    f = Figure(fontsize = 7)

    #Slider
    sl_i = Slider(f[4, 1:4], range = 1:length(dgms), startvalue = 1)
    x = sl_i.value

    # Persistence Diagrams
    pd_ax_a = Axis(f[1, 1], title = "Persistence codim = 2", xticks = 0:25:100, yticks = 0:25:100)
    pd_ax_b = Axis(f[1, 2], title = "Persistence codim = 3", xticks = 0:25:100, yticks = 0:25:100)
    dgm_points_2 = @lift([Point2f(dgms[$x][2][i,1], dgms[$x][2][i,2]) for i in 1:size(dgms[$x][2])[1]])
    dgm_points_3 = @lift([Point2f(dgms[$x][3][i,1], dgms[$x][3][i,2]) for i in 1:size(dgms[$x][3])[1]])

    ms = 5
    scatter!(pd_ax_a, dgm_points_2, color = :orange, markersize = ms)
    lines!(pd_ax_a, [0, 100], [0, 100], color=:black)
    scatter!(pd_ax_b, dgm_points_3, color = :green, markersize = ms)
    lines!(pd_ax_b, [0, 100], [0, 100], color=:black)

    #Energy and Theta
    theta_ax = Axis(f[2, 1], title = L"\Theta")
    E_ax = Axis(f[2, 2], title = L"F_{sol}")

    theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
    E_mark = @lift(Point2f($x, $@lift(Es[$x])))

    scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
    scatter!(theta_ax, theta_mark, color=:black, markersize = 10)

    scatter!(E_ax, xs, Es, color=:blue, markersize = ms)
    scatter!(E_ax, E_mark, color=:black, markersize = 10)


    # Configuration
    conf_ax = Axis3(f[1:2, 3:4])
    c1_points = @lift([Point3f(realizations[$x][1][1,i], realizations[$x][1][2,i], realizations[$x][1][3,i]) for i in 1:size(realizations[$x][1])[2]])
    c2_points = @lift([Point3f(realizations[$x][2][1,i], realizations[$x][2][2,i], realizations[$x][2][3,i]) for i in 1:size(realizations[$x][2])[2]])

    conf_ms = 10
    scatter!(conf_ax, c1_points, markersize = conf_ms)
    scatter!(conf_ax, c2_points, markersize = conf_ms)

    #Persistence Measures
    two_p_ax = Axis(f[3, 1], title = "codim 2 persistence")
    three_p_ax = Axis(f[3, 2], title = "codim 3 persistence")
    tp_ax = Axis(f[3, 3], title = "Total Persistence")

    tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
    two_p_mark = @lift(Point2f($x, $@lift(two_ps[$x])))
    three_p_mark = @lift(Point2f($x, $@lift(three_ps[$x])))

    scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
    scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

    scatter!(two_p_ax, xs, two_ps, color=:orange, markersize = ms)
    scatter!(two_p_ax, two_p_mark, color=:black, markersize = 10)

    scatter!(three_p_ax, xs, three_ps, color=:green, markersize = ms)
    scatter!(three_p_ax, three_p_mark, color=:black, markersize = 10)
    f
end

display(f)


GLMakie.Screen(...)

In [20]:
@load "../../Data/hpc_out/rwm_wp_2_6r7m/2_6r7m_protor_oj_0_0_os_0_85.jld2" input output
display(interactive_simulation_analysis(merge!(input, Dict("mol_type" => "6r7m")), output))

GLMakie.Screen(...)